In [2]:
import zmq
import json

class Robot():
    trades = []
    realizedPnL = []
    unrealizedPnL = []
    totalPnL = []
    position = []
    MaxPosition = 3
    currentPosition = 0
    canTrade = True
    uPnL = 0
    rPnL = 0
    tPnL = 0
    
    def __init__(self):
        return
    
    def calculatePnL(self,ask_value,bid_value):
        ask_value = float(ask_value)
        bid_value = float(bid_value)
        TotalBuyQty = 0
        TotalBuyCash = 0
        TotalSellQty = 0
        TotalSellCash = 0
        LeftOverPosition = 0
        VWAPSell = 0
        VWAPBuy = 0
        feePaid = 0

        for i in range(0, len(self.trades)):
            if self.trades[i][1] <= 0:
                TotalSellQty = TotalSellQty + abs(self.trades[i][1])
                TotalSellCash = TotalSellCash + abs(self.trades[i][1]) * self.trades[i][0]
            if self.trades[i][1] > 0:
                TotalBuyQty = TotalBuyQty + self.trades[i][1]
                TotalBuyCash = TotalBuyCash + abs(self.trades[i][1]) * self.trades[i][0]

        if TotalSellQty == 0:
            VWAPSell = 0
        else:
            VWAPSell = TotalSellCash / TotalSellQty

        if TotalBuyQty == 0:
            VWAPBuy = 0
        else:
            VWAPBuy = TotalBuyCash / TotalBuyQty

        LeftOverPosition = TotalBuyQty - TotalSellQty
    # Get realPnL
        self.rPnL = (VWAPSell - VWAPBuy) * min(TotalBuyQty, TotalSellQty)
    # Get unreal PnL
        if LeftOverPosition <= 0:
            self.uPnL = (VWAPSell - ask_value) * abs(LeftOverPosition)
        if LeftOverPosition > 0:
            self.uPnL = (VWAPBuy - bid_value) * abs(LeftOverPosition)
    # Append them to the PnL Arrays
        self.unrealizedPnL.append(self.uPnL)
        self.realizedPnL.append(self.rPnL)
        self.totalPnL.append(self.uPnL + self.rPnL)
        return self.totalPnL[-1]
    
    
    def processMessage(self,msg):
        
        #Recalculate unrealized PNL
        #pnl = currentPosition*(acquisitonPrice - currentPrice)
        #self.unrealizedPnL.append(pnl)
        if(self.canTrade and 'bidPrice' in msg.keys() and 'askPrice' in msg.keys()):
            if(float(msg['bidPrice']) <= 16205):
                desiredSize = 1
                if(abs(self.currentPosition + desiredSize) <= self.MaxPosition):
                    self.trades.append([msg['bidPrice'],desiredSize])
                    self.currentPosition = self.currentPosition + 1
                    #Calculate realized PnL
                    #pnl = realizedPnL[-1] + 1*(acquisitionPrice - currentPrice)
                    #self.calculatePnL(msg['askPrice'],msg['bidPrice'])
        
            if(float(msg['askPrice']) >= 16215):
                desiredSize = -1
                if(abs(self.currentPosition + desiredSize) <= self.MaxPosition):
                    self.trades.append([msg['askPrice'],desiredSize])
                    self.currentPosition = self.currentPosition - 1
                    #Calculated realized PnL
                    #pnl = realizedPnL[-1] - 1*(acquisitionPrice - currentPrice)
        self.calculatePnL(msg['askPrice'],msg['bidPrice'])
        self.position.append(self.currentPosition)
    
    def closePosition(self,msg):
        if(self.currentPosition < 0):
            desiredSize = abs(self.currentPosition)
            if(abs(self.currentPosition + desiredSize) < self.MaxPosition):
                self.trades.append([msg['bidPrice'],desiredSize])
                self.currentPosition = self.currentPosition + desiredSize
        if(self.currentPosition > 0):
            desiredSize = -1*self.currentPosition
            if(abs(self.currentPosition + desiredSize) < self.MaxPosition):
                self.trades.append([msg['askPrice'],desiredSize])
                self.currentPosition = self.currentPosition + desiredSize
        self.canTrade = False
        print(self.calculatePnL(msg['askPrice'],msg['bidPrice']))

In [ ]:
context = zmq.Context()
print("Connecting to server...")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:9000")

msg = {}
msg['tag'] = "CREATE_JOB"
msg['instruments'] = [{'exchange':'FORTS','instrument':'SBRF-6.17'},{'exchange':'FORTS','instrument':'VTBR-6.17'}]
msg['dateStart'] = 20170922
msg['dateEnd'] = 20170922
socket.send_string(str(msg))
x = socket.recv()
print(x)
y = dict(json.loads(x))

socket2 = context.socket(zmq.SUB)
socket2.setsockopt_string(zmq.SUBSCRIBE,"")
socket2.connect("tcp://localhost:"+ str(y['port']))

msg = {}
msg['tag'] = "START_JOB"
msg['uuid'] = str(y['id'])
socket.send_string(str(msg))
x = socket.recv()
print(x)

robot = Robot()
x = None
i = 0
Indicator1 = []
Indicator2 = []

while(x != b'END_OF_TRANSMISSION'):
    x = socket2.recv()
    try:
        y = dict(json.loads(x))
        if(i >= 300):
            robot.processMessage(y)
        if(y['instrument'] == 'SBRF-6.17'):
            Indicator1.append(y['bidPrice'])
            Indicator2.append(Indicator2[-1])
        if(y['instrument'] == 'VTBR-6.17'):
            Indicator1.append(Indicator1[-1])
            Indicator2.append(y['bidPrice'])
        i = i + 1
    except:
        print("Exception")
        print(x)
        
print("Final PnL:" + str(robot.realizedPnL[-1]))

Connecting to server...
b'{"port":2000,"id":"761ea841-cac4-41c5-862f-2ae26894f998"}'
b'OK'
Exception
b'{"tag":"START_OF_STREAM"}'
Exception
b'{"bidSize":1,"instrument":"VTBR-6.17","tag":"Q","bidPrice":6250,"ts":1493394565510568800}'
Exception
b'{"bidSize":1,"instrument":"VTBR-6.17","tag":"Q","bidPrice":6270,"ts":1493394566332940400}'
Exception
b'{"askPrice":7170,"bidSize":1,"instrument":"VTBR-6.17","tag":"Q","bidPrice":6270,"askSize":1,"ts":1493394566333661300}'
Exception
b'{"askPrice":6870,"bidSize":1,"instrument":"VTBR-6.17","tag":"Q","bidPrice":6270,"askSize":1,"ts":1493394566334350500}'
Exception
b'{"bidSize":1,"instrument":"SBRF-6.17","tag":"Q","bidPrice":14500,"ts":1493394566385915600}'
Exception
b'{"askPrice":16813,"bidSize":1,"instrument":"SBRF-6.17","tag":"Q","bidPrice":14500,"askSize":3,"ts":1493394566423081400}'


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(robot.totalPnL,c='green')
ax2 = ax1.twinx()
ax2.plot(robot.position,c='red')

In [ ]:
plt.plot(Indicator1[200:])
plt.plot(Indicator2[200:])

In [ ]:
print(robot.trades)